In [3]:
import pandas as pd 
import numpy as np

In [4]:
import pandas as pd 
import numpy as np

def get_csv(path):
    df = pd.read_csv(path)
    return df

#create all data frames
aisle = get_csv('./instacart_data/aisles.csv')
depart = get_csv('./instacart_data/departments.csv')
products = get_csv('./instacart_data/products.csv')
order_train = get_csv('./instacart_data/order_products__train.csv')
order_prior = get_csv('./instacart_data/order_products__prior.csv')
orders = get_csv('./instacart_data/orders.csv') #3421083 orders

In [147]:
orders

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
...,...,...,...,...,...,...,...
3421078,2266710,206209,prior,10,5,18,29.0
3421079,1854736,206209,prior,11,4,10,30.0
3421080,626363,206209,prior,12,1,12,18.0
3421081,2977660,206209,prior,13,1,12,7.0


In [151]:
#confirm users are in either test or train set
num = orders.groupby('eval_set').agg({'user_id':'nunique'})
num

,user_id
eval_set,
prior,206209
test,75000
train,131209


In [20]:
#train set
order_train

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
...,...,...,...,...
1384612,3421063,14233,3,1
1384613,3421063,35548,4,1
1384614,3421070,35951,1,1
1384615,3421070,16953,2,1


In [7]:
#bring in orders full data to order_train
train = pd.merge(order_train, orders, how='left')
train

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,49302,1,1,112108,train,4,4,10,9.0
1,1,11109,2,1,112108,train,4,4,10,9.0
2,1,10246,3,0,112108,train,4,4,10,9.0
3,1,49683,4,0,112108,train,4,4,10,9.0
4,1,43633,5,1,112108,train,4,4,10,9.0
...,...,...,...,...,...,...,...,...,...,...
1384612,3421063,14233,3,1,169679,train,30,0,10,4.0
1384613,3421063,35548,4,1,169679,train,30,0,10,4.0
1384614,3421070,35951,1,1,139822,train,15,6,10,8.0
1384615,3421070,16953,2,1,139822,train,15,6,10,8.0


In [21]:
#find previous order number and use that to train
#maybe use train as my test?
train['prev'] = train['order_number'] - 1

In [33]:
#make new column
train['prev2'] = train['order_number'] - 2

In [36]:
#create
train

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,prev,prev2
0,1,49302,1,1,112108,train,4,4,10,9.0,3,2
1,1,11109,2,1,112108,train,4,4,10,9.0,3,2
2,1,10246,3,0,112108,train,4,4,10,9.0,3,2
3,1,49683,4,0,112108,train,4,4,10,9.0,3,2
4,1,43633,5,1,112108,train,4,4,10,9.0,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...
1384612,3421063,14233,3,1,169679,train,30,0,10,4.0,29,28
1384613,3421063,35548,4,1,169679,train,30,0,10,4.0,29,28
1384614,3421070,35951,1,1,139822,train,15,6,10,8.0,14,13
1384615,3421070,16953,2,1,139822,train,15,6,10,8.0,14,13


In [30]:
#look at user_id 66411
#yes, has made 97 orders
user66411 = orders[orders['user_id'] == 66411]
user66411['order_number'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97])

In [29]:
#look at user_id 38495
#yes, has made 97 orders
user38495 = orders[orders['user_id'] == 38495]
user38495['order_number'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97])

In [40]:
#none of test users are in train users
set(test['user_id']) == set(train_df['user_id'])

False

In [11]:
#look at user 3 who is in test dataframe
orders[orders["user_id"] == 3]

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
26,1374495,3,prior,1,1,14,NaN
27,444309,3,prior,2,3,19,9.0
28,3002854,3,prior,3,3,16,21.0
29,2037211,3,prior,4,2,18,20.0
30,2710558,3,prior,5,0,17,12.0
31,1972919,3,prior,6,0,16,7.0
32,1839752,3,prior,7,0,15,7.0
33,3225766,3,prior,8,0,17,7.0
34,3160850,3,prior,9,0,16,7.0
35,676467,3,prior,10,3,16,17.0


In [ ]:
#make my own test df

In [130]:
#make test df from orders
order_test = orders[orders['eval_set'] == "test"].copy()
order_test

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
38,2774568,3,test,13,5,15,11.0
44,329954,4,test,6,3,12,30.0
53,1528013,6,test,4,3,16,22.0
96,1376945,11,test,8,6,11,8.0
102,1356845,12,test,6,1,20,30.0
...,...,...,...,...,...,...,...
3420918,2728930,206202,test,23,2,17,6.0
3420929,350108,206204,test,5,4,14,14.0
3421001,1043943,206206,test,68,0,20,0.0
3421018,2821651,206207,test,17,2,13,14.0


In [157]:
#get list of test users and goal is to get their previous order
#create column of prev order num for these customers
# order_test['prev'] = order_test['order_number'] - 1

In [153]:
#get df I want
order_test = order_test[['user_id', 'prev']]

In [158]:
order_test

,user_id,prev
38,3,12
44,4,5
53,6,3
96,11,7
102,12,5
...,...,...
3420918,206202,22
3420929,206204,4
3421001,206206,67
3421018,206207,16


In [155]:
#code to merge test users to previous order number
merged_test = order_test.merge(orders, how='inner', left_on=["user_id", "prev"], right_on=["user_id","order_number"])

In [136]:
#correct merged test
merged_test[['user_id', 'prev', 'order_id', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order']]

,user_id,prev,order_id,order_dow,order_hour_of_day,days_since_prior_order
0,3,12,1402502,1,15,15.0
1,4,5,2557754,5,13,0.0
2,6,3,998866,2,18,12.0
3,11,7,1468214,5,9,30.0
4,12,5,221248,1,9,30.0
...,...,...,...,...,...,...
74995,206202,22,1112560,3,14,6.0
74996,206204,4,1380497,4,15,25.0
74997,206206,67,1904200,0,13,11.0
74998,206207,16,1005822,2,7,18.0


In [140]:
#connect order_id to prior df to get itemized 
final_test = pd.merge(merged_test, order_prior, on="order_id")

In [144]:
final_test[['user_id', 'prev', 'order_id', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order', 'product_id',
       'add_to_cart_order', 'reordered']]

,user_id,prev,order_id,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,3,12,1402502,1,15,15.0,39190,1,1
1,3,12,1402502,1,15,15.0,18599,2,1
2,3,12,1402502,1,15,15.0,23650,3,1
3,3,12,1402502,1,15,15.0,21903,4,1
4,3,12,1402502,1,15,15.0,47766,5,1
...,...,...,...,...,...,...,...,...,...
780795,206208,49,1882108,1,22,7.0,2643,13,1
780796,206208,49,1882108,1,22,7.0,21405,14,0
780797,206208,49,1882108,1,22,7.0,7632,15,0
780798,206208,49,1882108,1,22,7.0,1942,16,0
